In [19]:
import numpy as np
import scipy.optimize as opt


In [4]:
# Given Data
S_t0 = np.array([1.755, 1.17, 0.975])  # Asset prices at time t0
A = np.array([[1, 0.5, 1], 
              [2, 1.75, 1], 
              [3, 1.5, 1]])  # Price matrix at time T
p_real = np.array([0.30, 0.15, 0.55])  # Real-world probabilities

In [5]:
# Step 1: Check Market Completeness (A must be square and full rank)
rank_A = np.linalg.matrix_rank(A)
market_complete = (rank_A == A.shape[1])  # True if rank(A) = number of assets\
print(f"Market is Complete: {market_complete}")

Market is Complete: True


In [18]:
Inv_A = np.linalg.inv(A)
Inv_A


array([[-0.16666667, -0.66666667,  0.83333333],
       [-0.66666667,  1.33333333, -0.66666667],
       [ 1.5       ,  0.        , -0.5       ]])

In [13]:
qt= 1.0256 * S_t0 * Inv_A
qt

array([[-0.299988, -0.799968,  0.8333  ],
       [-1.199952,  1.599936, -0.66664 ],
       [ 2.699892,  0.      , -0.49998 ]])

In [11]:
# Step 2: Compute Risk-Neutral Probabilities (Q)
# Solve for Q such that S_t0 = Q^T * A / (1 + risk-free rate)
# Assume risk-free rate i = 0 for simplicity

q = np.linalg.solve(A.T, S_t0)  # Solving A^T * q = S_t0
q_positive = np.all(q > 0)  # Market is arbitrage-free if all Q values are positive

print(f"Market is Arbitrage-Free: {q_positive}")
print(f"Risk-Neutral Probabilities (Q): {np.round(q,4)}")

Market is Arbitrage-Free: True
Risk-Neutral Probabilities (Q): [0.39  0.39  0.195]


In [8]:
# Step 3: Solve for the Optimal Portfolio (Utility Maximization)
W_0 = 50  # Initial Wealth
# Step 2: Define the Utility Maximization Problem
utility_function = lambda W: -np.sum(p_real * np.log(4 * W + 1))  # Expected utility to maximize

# Constraint: Discounted martingale-weighted wealth equals initial wealth
constraint = {'type': 'eq', 'fun': lambda W: np.sum(q * W) - W_0}

# Bounds: Wealth in each scenario must be positive
bounds = [(0, None) for _ in range(A.shape[0])]

# Initial Guess: Equally distributed wealth
W_init = np.ones(A.shape[0]) * (W_0 / A.shape[0])

# Solve Optimization Problem
result = opt.minimize(utility_function, W_init, constraints=constraint, bounds=bounds)
W_T_optimal = result.x  # Optimal wealth in each scenario

# Step 3: Compute Portfolio Weights α = A^(-1) * W_T
alpha_optimal = np.linalg.solve(A, W_T_optimal)

# Display Results
print(f"Market is Arbitrage-Free: {q_positive}")
print(f"Risk-Neutral Probabilities (Q): {q}")
print(f"Optimal Wealth in Each Scenario (W_T): {W_T_optimal}")
print(f"Optimal Portfolio Weights (α): {alpha_optimal}")

Market is Arbitrage-Free: True
Risk-Neutral Probabilities (Q): [0.39  0.39  0.195]
Optimal Wealth in Each Scenario (W_T): [ 41.48579585  17.91441138 137.60984114]
Optimal Portfolio Weights (α): [ 95.81762739 -95.51120949  -6.5762268 ]
